# Energy billing: Data generation

This notebook aggregates and generates the user energy consumption and/or production data used in the experimental verification of the Energy Billing project.

The data aggregated in this file, uses data generated by three other notebooks:
- [consumption](./consumption/consumption.ipynb), 
- [pv](./pv/pv.ipynb), and 
- [wind](./wind/wind.ipynb).

The code in this library was inspired by [the notebook in this repository](https://github.com/PeijieZ/Billing-Models-for-Electricity-Trading-Markets).

In [ ]:
import pandas as pd
import random

In [ ]:
# Generation settings

# Generate for...
DAYS = 31
TIMESTEPS_PER_DAY = 24
TIMESTEPS = DAYS * TIMESTEPS_PER_DAY
HOUSEHOLDS = 100

# Generate using ...
DECIMAL_PRECISION = 4

## Load data

In [ ]:
# Consumption data
CONSUMPTION = pd.read_json(f'consumption/out/consumption_{DAYS}_days.json')
CONSUMPTION

In [ ]:
# PhotoVoltaic production data
PV = pd.read_json(f'pv/out/solarpower_{DAYS}_days.json')
PV

In [ ]:
# WindTurbine production data
WT = pd.read_json(f'wind/out/windpower_{DAYS}_days.json')
WT

## Combine data

### Initialize data structure

In [ ]:
data = [
    [
        {}
        for hh in range(HOUSEHOLDS)
    ]
    for ts in range (TIMESTEPS)
]

### Aggregate consumption and production data

In [ ]:
for ts_idx, ts in enumerate(data):

    pv_ts_data = PV.iloc[ts_idx].T
    wt_ts_data = WT.iloc[ts_idx].T

    # Aggregate data for this timestep
    for hh, stats in enumerate(ts):

        # Consumption data
        consumption = CONSUMPTION[hh]
        consumption = consumption[ts_idx]
        stats['consumption'] = consumption

        # WT data: add for 0 - 9
        if hh < 10:
            wt = wt_ts_data.iloc[hh]
        else:
            wt = 0
        stats['wind'] = wt

        # PV data: add for 5 - 29
        if 5 <= hh < 30:
            pv = pv_ts_data.iloc[hh - 5]    
        else:
            pv = 0
        stats['pv'] = pv

        # Supply
        stats['supply'] = supply = wt + pv

        # Consumption/production profile
        cp_profile = supply - consumption
        stats['cp profile'] = round(cp_profile, DECIMAL_PRECISION)

        ts[hh] = stats

df = pd.DataFrame(data[16]).T
df

# Add further statistics

### Generate 'predictions' of a user's consumption

In [ ]:
predictions_set =  15 * [("perfect",(0.00, 0.00))] \
                 + 90 * [("good",   (0.01, 0.10))] \
                 + 30 * [("ok",     (0.11, 0.20))] \
                 + 15 * [("bad",    (0.21, 1.00))]
predictions_set = predictions_set[:HOUSEHOLDS]
assert len(predictions_set) == HOUSEHOLDS
random.shuffle(predictions_set)


for ts_idx, ts in enumerate(data):
    for stats, prediction_settings in zip(ts, predictions_set):
        category, interval = prediction_settings
        
        # predict based on
        # cp_profile = stats['cp profile']
        supply = stats['supply']
        consumption = stats['consumption']

        # Determine errors
        supply_error = supply * random.uniform(*interval) * random.choice([-1, 1])
        consumption_error = consumption * random.uniform(*interval) * random.choice([-1, 1])

        pred_supply = round(supply + supply_error, DECIMAL_PRECISION)
        pred_consumption = round(consumption + consumption_error, DECIMAL_PRECISION)

        stats['prediction algorithm'] = category
        stats['consumption prediction'] = abs(pred_consumption)
        stats['supply prediction'] = abs(pred_supply)

df = pd.DataFrame(data[13]).T
df

### Add supplier information

In [ ]:
# Generate random supplier information
supplier_info_set =  45 * [("supplier1", 0.200)] \
                   + 45 * [("supplier2", 0.205)] \
                   + 30 * [("supplier3", 0.210)] \
                   + 15 * [("supplier4", 0.215)] \
                   + 15 * [("supplier3", 0.220)]
random.shuffle(supplier_info_set)
supplier_info_set = supplier_info_set[:HOUSEHOLDS]
assert len(supplier_info_set) == HOUSEHOLDS

# Assign a supplier to each consumer
for ts in data:
    for stats, supplier_info in zip(ts, supplier_info_set):
        supplier, rate = supplier_info
        stats['supplier'] = supplier
        stats['retail price'] = rate

df = pd.DataFrame(data[16]).T
df

### Run auction

In [ ]:
BUYER_PRICE_INTERVAL = 0.05, 0.19
SELLER_PRICE_INTERVAL = 0.06, 0.20
TRADING_PRICE = 0.11
FEED_IN_TARIF = 0.05

def get_random_price(interval):
    return round(random.uniform(*interval), 2)

for ts_idx, ts in enumerate(data):

    # Determine role for this round
    buyers, sellers = [], []
    for hh_stats in ts:

        pred_supply = hh_stats['supply prediction']
        pred_consumption = hh_stats['consumption prediction']
        pred_effective = pred_supply - pred_consumption
        
        if pred_effective >= 0:
            role = "seller"
            sellers.append(hh_stats)
            hh_stats['consumption promise'] = 0
            hh_stats['supply promise'] = hh_stats['supply prediction'] - hh_stats['consumption prediction']
        else:
            role = "buyer"
            buyers.append(hh_stats)
            hh_stats['consumption promise'] = hh_stats['consumption prediction'] - hh_stats['supply prediction'] 
            hh_stats['supply promise'] = 0

        hh_stats['actual consumption'] = max(0, hh_stats['consumption'] - hh_stats['supply'])
        hh_stats['actual supply'] = max(0, hh_stats['supply'] - hh_stats['consumption'])

        hh_stats['role'] = role

    # Give buyers a random trading price
    for buyer in buyers:
        price = get_random_price(BUYER_PRICE_INTERVAL)
        buyer['trading price'] = price

        # Accept if greater than trading price
        if price >= TRADING_PRICE:
            buyer['bid'] = "accepted"
        else:
            buyer['bid'] = "rejected"

    # Give sellers a random trading price
    for seller in sellers:
        price = get_random_price(SELLER_PRICE_INTERVAL)
        seller['trading price'] = price

        # Accept if smaller than trading price
        if price <= TRADING_PRICE:
            seller['bid'] = "accepted"
        else:
            seller['bid'] = "rejected"

# Record number of consumers and prosumers
for ts_idx, ts in enumerate(data):
    accepted_sellers = list(filter(lambda x: x['bid'] == "accepted" and x['role'] == "seller", ts))
    nr_accepted_sellers = len(accepted_sellers)
    accepted_buyers = list(filter(lambda x: x['bid'] == "accepted" and x['role'] == "buyer", ts))
    nr_accepted_buyers = len(accepted_buyers)

    for hh_stats in ts:
        hh_stats['total acc. prosumers'] = max(nr_accepted_sellers, 1) ## prevent division by zero problems; should not impact results
        hh_stats['total acc. consumers'] = max(nr_accepted_buyers, 1) ## prevent division by zero problems; should not impact results

# Set feed-in tarif
for ts in data:
    for hh in ts:
        hh['feed in tarif'] = FEED_IN_TARIF

df = pd.DataFrame(data[14]).T
df

## Total deviation

In [ ]:
# Get deviation of a user
def get_deviation(stats):
    if stats['role'] == "buyer":
        consumption_prediction = stats['consumption prediction'] - stats['supply prediction']
        effective_consumption = stats['consumption'] - stats['supply']
        deviation = effective_consumption - consumption_prediction

    elif stats['role'] == "seller":
        supply_prediction = stats['supply prediction'] - stats['consumption prediction']
        effective_supply = stats['supply'] - stats['consumption']
        deviation = effective_supply - supply_prediction

    return deviation

# Compute total deviation among accepted buyers and sellers
def get_total_deviation(data):

    # Get buyer deviation
    buyers = filter(lambda x: x['role'] == "buyer", data)
    accepted_buyers = list(filter(lambda x: x['bid'] == "accepted", buyers))
    
    buyer_deviation = 0
    for buyer in accepted_buyers:
        buyer_deviation += get_deviation(buyer)

    sellers = filter(lambda x: x['role'] == "seller", data)
    accepted_sellers = list(filter(lambda x: x['bid'] == "accepted", sellers))

    seller_deviation = 0
    for seller in accepted_sellers:
        seller_deviation += get_deviation(seller)

    return seller_deviation - buyer_deviation


# Compute bill for everybody
for ts_idx, ts in enumerate(data):
    total_deviation = get_total_deviation(ts)

    for hh_stats in ts:
        hh_stats['total deviation'] = total_deviation
        hh_stats['deviation'] = get_deviation(hh_stats)
        hh_stats['accepted'] = 1 if hh_stats['bid'] == "accepted" else 0
        # hh_stats['accepted'] = 0

In [ ]:
# Overwrite trading price to be TRADING_PRICE
for ts in data:
    for hh_stats in ts:
        hh_stats['trading price'] = TRADING_PRICE

In [ ]:
# Compute bill and reward
for ts_idx, ts in enumerate(data):
    for hh_stats in ts:
        total_dev = hh_stats['total deviation']

        if hh_stats['accepted'] == 1:
            # P2P trading
            bill = hh_stats['actual consumption'] * TRADING_PRICE
            reward = hh_stats['actual supply'] * TRADING_PRICE

            if total_dev < 0 and hh_stats['deviation'] > 0 and hh_stats['role'] == "buyer":
                bill += total_dev / hh_stats['total acc. consumers'] * (hh_stats['retail price'] - TRADING_PRICE)

            if total_dev > 0 and hh_stats['deviation'] > 0 and hh_stats['role'] == "seller":
                reward += total_dev / hh_stats['total acc. prosumers'] * (FEED_IN_TARIF - TRADING_PRICE)

        else:
            # No p2p trading
            bill = hh_stats['actual consumption'] * hh_stats['retail price']
            reward = hh_stats['actual supply'] * FEED_IN_TARIF

        hh_stats['bill'] = bill
        hh_stats['reward'] = reward

## Export data

In [ ]:
# Convert from per-day to per-client dataslices
user_slices = []
for u_idx in range(HOUSEHOLDS):
    user_slice = []
    for ts in data:
        user_slice.append(ts[u_idx])
    user_slices.append(user_slice)

user_frames = []
for slice in user_slices:
    df = pd.DataFrame(slice).T
    user_frames.append(df)
    
user_frames[1]

In [ ]:
# strip unused values

user_frames = [
    df.drop([
        'consumption',
        'supply',
        'feed in tarif',
        'wind',
        'pv',
        'cp profile',
        'prediction algorithm',
        'supplier',
        'role',
        'trading price',
        'total acc. consumers',
        'total acc. prosumers',
        'total deviation',
        'consumption prediction',
        'supply prediction',
        'bid'
    ])
    for df in user_frames
]

user_frames[1]

In [ ]:
# Export individual user data
from pathlib import Path
data_dir = Path(f"./data/{TIMESTEPS}_ts_{HOUSEHOLDS}_clients/")
data_dir.mkdir(exist_ok=True)
for idx, user in enumerate(user_frames):
    user = user.T.iloc[:TIMESTEPS].T    
    user.to_csv(data_dir / f"user_{idx}.csv")

# Export bill context data
context = pd.DataFrame(user_slices[0]).T
context = context.loc[[
    'feed in tarif',
    'trading price',
    'total acc. consumers',
    'total acc. prosumers',
    'total deviation']
].T.iloc[:TIMESTEPS].T
context.to_csv(data_dir / "context.csv")

## Future work

### Improved auction

In [ ]:
# Introduce the buyer / seller statistic

BUYER_PRICE_LOW, BUYER_PRICE_HIGH = 0.05, 0.19
BUYER_VOL_GRADIENT = 2

def get_buyer_bid(pred_consumption_vol) -> float:
    """
    Scale price linearly with consumption,
    increasing as volume increases.
    """
    scale = min(pred_consumption_vol / BUYER_VOL_GRADIENT, 1)
    bid = (1-scale) * BUYER_PRICE_LOW + scale * BUYER_PRICE_HIGH
    return round(bid, 2)


SELLER_PRICE_LOW, SELLER_PRICE_HIGH = 0.06, 0.20
SELLER_VOL_GRADIENT = 3.5

def get_seller_offer(pred_production_vol) -> float:
    """
    Scale price linearly with production,
    decreasing as volume increases.
    """
    scale = min(pred_production_vol / SELLER_VOL_GRADIENT, 1)
    offer = scale * SELLER_PRICE_LOW + (1-scale) * SELLER_PRICE_HIGH
    return round(offer, 2)


for ts_idx, ts in enumerate(data):

    # Determine role for this round
    buyers, sellers = [], []
    for hh_stats in ts:
        prediction = hh_stats['prediction']
        
        if prediction >= 0:
            role = "seller"
            sellers.append(hh_stats)
        else:
            role = "buyer"
            buyers.append(hh_stats)

        hh_stats['role'] = role

    # Determine price offer
    offers, bids = [], []
    for hh_id, hh_stats in enumerate(ts):
        prediction = abs(hh_stats['prediction'])

        if hh_stats in buyers:
            pred_cons_vol = abs(prediction)
            price = get_buyer_bid(pred_cons_vol)
            bids.append((hh_id, price, pred_cons_vol))
        elif hh_stats in sellers:
            pred_prod_vol = abs(prediction)
            price = get_seller_offer(pred_prod_vol)
            offers.append((hh_id, price, pred_prod_vol))
        else:
            raise Exception("woops, something went wrong here.")
        hh_stats['offer'] = price
    
    # Run the auction
    assert len(offers) + len(bids) == HOUSEHOLDS

    # Sort offers and bids: lowest to highest price AND largest to smallest volume
    offers.sort(key=lambda x: (x[1], -x[2]))
    bids.sort(key=lambda x: (x[1], -x[2]))

    while len(offers) > 0 and len(bids) > 0:
        # Try to match cheapest offer with highest bid
        highest_offer = offers.pop()
        highest_bid = bids.pop()
        seller_id, offer, seller_vol = highest_offer
        buyer_id, bid, buyer_vol = highest_bid

        if offer > bid:
            # Reject seller
            ts[seller_id]['bid'] = "rejected"

            # Return bidder to pool
            bids.append(highest_bid)

            continue

        if seller_vol >= buyer_vol:
            # Buy the buyer out
            ts[buyer_id]['bid'] = "accepted"

            # Return the remaining offer to pool
            remaining = seller_vol - buyer_vol
            if remaining > 0:
                rem_offer = (seller_id, offer, remaining)
                offers.append(rem_offer)

        if buyer_vol >= seller_vol:
            # Buy the seller out
            ts[seller_id]['bid'] = "accepted"

            # Return remaining bid to pool
            remaining = buyer_vol - seller_vol
            if remaining > 0:
                rem_bid = (buyer_id, bid, remaining)
                bids.append(rem_bid)
        
    # Set remaining bids/offers to rejected
    for seller_id, _, _ in offers:
        ts[seller_id]['bid'] = "rejected"
    for buyer_id, _, _ in bids:
        ts[buyer_id]['bid'] = "rejected"

df = pd.DataFrame(data[142]).T
df